### 기상청 날씨 데이터 파싱
* BeautifulSoup의 find(), find_all() 함수 사용
* 파싱한 데이터를 dict와 list 에 저장
* json file로 저장하기

In [11]:
import requests 
from bs4 import BeautifulSoup

url = 'http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp'
res = requests.get(url)
print(res.status_code)
html = res.text
soup = BeautifulSoup(html, 'html.parser')
#print(soup)

title = soup.find('title')
print(type(title), title, title.text)

titles = soup.find_all('title')
print(type(titles), titles, titles[0].text)
for item in titles:
    print(type(item.text), item.text)
    print(type(item.string), item.string)

200
<class 'bs4.element.Tag'> <title>기상청 육상 중기예보</title> 기상청 육상 중기예보
<class 'bs4.element.ResultSet'> [<title>기상청 육상 중기예보</title>, <title>전국 육상 중기예보 - 2021년 09월 28일 (화)요일 06:00 발표</title>, <title>전국 육상중기예보</title>] 기상청 육상 중기예보
<class 'str'> 기상청 육상 중기예보
<class 'bs4.element.NavigableString'> 기상청 육상 중기예보
<class 'str'> 전국 육상 중기예보 - 2021년 09월 28일 (화)요일 06:00 발표
<class 'bs4.element.NavigableString'> 전국 육상 중기예보 - 2021년 09월 28일 (화)요일 06:00 발표
<class 'str'> 전국 육상중기예보
<class 'bs4.element.NavigableString'> 전국 육상중기예보


In [15]:
link = soup.select_one('title')
print(type(link), link, link.string)

title_all = soup.select('title')
print(type(title_all), title_all)

<class 'bs4.element.Tag'> <title>기상청 육상 중기예보</title> 기상청 육상 중기예보
<class 'bs4.element.ResultSet'> [<title>기상청 육상 중기예보</title>, <title>전국 육상 중기예보 - 2021년 09월 28일 (화)요일 06:00 발표</title>, <title>전국 육상중기예보</title>]


In [17]:
# link 태그 text 못 가지고 옴
link = soup.find('channel').find('link')
print(link)

<link/>


In [26]:
location = soup.find('location', attrs={"wl_ver":"3"})
province = location.find('province').text
print(province)
city = location.find('city').text
print(city)
for data in location.find_all('data'):
    print(data.find('mode').text, data.find('wf').text)

서울ㆍ인천ㆍ경기도
서울
A02 구름많음
A02 맑음
A02 구름많음
A02 맑음
A02 구름많음
A02 구름많음
A02 흐림
A02 흐림
A02 흐림
A02 흐림
A01 흐리고 비
A01 흐림
A01 흐림


### 전국날씨 데이터를 dict와 list 저장하기


In [31]:
import requests 
from bs4 import BeautifulSoup

url = 'http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp'
res = requests.get(url)
print(res.status_code)
html = res.text
soup = BeautifulSoup(html, 'html.parser')

#41개의 location 정보를 저장할 list 선언
location_list = list()
location_tags = soup.find_all('location', attrs={"wl_ver":"3"})
print(len(location_tags))
for location in location_tags:
    #1개의 location 정보를 저장할 dict 선언
    location_dict = dict()
    location_dict['province'] = location.find('province').text
    location_dict['city'] = location.find('city').text
    print(location_dict)

200
41
{'province': '서울ㆍ인천ㆍ경기도', 'city': '서울'}
{'province': '서울ㆍ인천ㆍ경기도', 'city': '인천'}
{'province': '서울ㆍ인천ㆍ경기도', 'city': '수원'}
{'province': '서울ㆍ인천ㆍ경기도', 'city': '파주'}
{'province': '서울ㆍ인천ㆍ경기도', 'city': '이천'}
{'province': '서울ㆍ인천ㆍ경기도', 'city': '평택'}
{'province': '강원도영서', 'city': '춘천'}
{'province': '강원도영서', 'city': '원주'}
{'province': '강원도영동', 'city': '강릉'}
{'province': '대전ㆍ세종ㆍ충청남도', 'city': '대전'}
{'province': '대전ㆍ세종ㆍ충청남도', 'city': '세종'}
{'province': '대전ㆍ세종ㆍ충청남도', 'city': '홍성'}
{'province': '충청북도', 'city': '청주'}
{'province': '충청북도', 'city': '충주'}
{'province': '충청북도', 'city': '영동'}
{'province': '광주ㆍ전라남도', 'city': '광주'}
{'province': '광주ㆍ전라남도', 'city': '목포'}
{'province': '광주ㆍ전라남도', 'city': '여수'}
{'province': '광주ㆍ전라남도', 'city': '순천'}
{'province': '광주ㆍ전라남도', 'city': '광양'}
{'province': '광주ㆍ전라남도', 'city': '나주'}
{'province': '전라북도', 'city': '전주'}
{'province': '전라북도', 'city': '군산'}
{'province': '전라북도', 'city': '정읍'}
{'province': '전라북도', 'city': '남원'}
{'province': '전라북도', 'city': '고창'}
{'province': '